In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df = pd.read_csv('train.csv', header=0, sep=',', quotechar='"', on_bad_lines='skip') #on_bad_lines='skip'
print(df.head)
# df_test = pd.read_csv('test.csv', header=0, sep=',', quotechar='"', on_bad_lines='skip') #on_bad_lines='skip'
# print(df_test.head)

<bound method NDFrame.head of            text_id                                          full_text  \
0     0016926B079C  I think that students would benefit from learn...   
1     0022683E9EA5  When a problem is a change you have to let it ...   
2     00299B378633  Dear, Principal\n\nIf u change the school poli...   
3     003885A45F42  The best time in life is when you become yours...   
4     0049B1DF5CCC  Small act of kindness can impact in other peop...   
...            ...                                                ...   
3906  FFD29828A873  I believe using cellphones in class for educat...   
3907  FFD9A83B0849  Working alone, students do not have to argue w...   
3908  FFDC4011AC9C  "A problem is a chance for you to do your best...   
3909  FFE16D704B16  Many people disagree with Albert Schweitzer's ...   
3910  FFED00D6E0BD  Do you think that failure is the main thing fo...   

      cohesion  syntax  vocabulary  phraseology  grammar  conventions  
0          3.5     3.

In [3]:
df = df.drop(columns=['cohesion','vocabulary','phraseology','grammar','conventions'])
# df_test = df_test.drop(columns=['cohesion','vocabulary','phraseology','grammar','conventions'])

df['syntax'] = df['syntax'] * 2

print(list(df.columns))
print(df['syntax'].head)
print(df.shape)

['text_id', 'full_text', 'syntax']
<bound method NDFrame.head of 0       7.0
1       5.0
2       7.0
3       9.0
4       6.0
       ... 
3906    6.0
3907    8.0
3908    6.0
3909    9.0
3910    5.0
Name: syntax, Length: 3911, dtype: float64>
(3911, 3)


In [4]:
print(df['syntax'].describe())

count    3911.000000
mean        6.056507
std         1.288798
min         2.000000
25%         5.000000
50%         6.000000
75%         7.000000
max        10.000000
Name: syntax, dtype: float64


# Preprocessing

In [5]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
import contractions

df['full_text'] = df['full_text'].str.lower()
df['full_text'] = df['full_text'].str.lstrip()
df['full_text'] = df['full_text'].str.rstrip()
df['full_text'] = df['full_text'].str.replace(' +', ' ') #remove duplicate space - problem: might miss

df['full_text'] = df['full_text'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))

df['full_text'] = df['full_text'].str.replace(r'[^a-zA-Z\d\s:]', '') #removed special characters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


In [7]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

df['full_text'] = df['full_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
print(df.head(20))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


         text_id                                          full_text  syntax
0   0016926B079C  think students would benefit learning homebeca...     7.0
1   0022683E9EA5  problem change let best matter happening chang...     5.0
2   00299B378633  dear principal change school policy grade b av...     7.0
3   003885A45F42  best time life become agree greatest accomplis...     9.0
4   0049B1DF5CCC  small act kindness impact people change people...     6.0
5   004AC288D833  dear principal school community center reasons...     8.0
6   005661280443  imagine could prove people good problem solver...     8.0
7   008DDDDD8E8D  think good idea estudnets commit career young ...     5.0
8   009BCCC61C2A  positive attitude key success agree anything l...     6.0
9   009F4E9310CB  asking one person advice help take strong choi...     6.0
10  00B21F9B726F  think good idea students commit career young a...     7.0
11  00BCADB373EF  positive attitude key success many people posi...     6.0
12  00D28152

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit

train, test = train_test_split(df, test_size=0.2, random_state=2, shuffle=True, stratify=df['syntax'].values.astype(int))

train_labels = train['syntax'].values.astype(int).tolist()
test_labels = test['syntax'].values.astype(int).tolist()

target_names = df['syntax'].unique() / 2
target_names = np.sort(target_names)
target_names_list = [str(i) for i in target_names]


# Gather TFIDF values

In [ ]:

vector = TfidfVectorizer(analyzer = 'word') #, ngram_range = (2,2)
tfidf_train = vector.fit_transform(train['full_text'])
tfidf_features = vector.get_feature_names_out()
tfidf_test = vector.transform(test['full_text'])

# Begin Running Linear Models on Data

In [ ]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

perceptron_classifier = Perceptron(random_state=0) #max_iter=200

perceptron_classifier.fit(tfidf_train, train_labels)
y_pred = perceptron_classifier.predict(tfidf_test)

target_names = df['syntax'].unique() / 2
target_names = np.sort(target_names)
target_names_list = [str(i) for i in target_names]

# print(target_names_list, "target_names", len(target_names_list))
print(classification_report(test_labels, y_pred, target_names=target_names_list))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         1.5       0.00      0.00      0.00         6
         2.0       0.22      0.13      0.17        82
         2.5       0.27      0.23      0.24       168
         3.0       0.33      0.36      0.35       250
         3.5       0.24      0.32      0.27       174
         4.0       0.18      0.13      0.15        78
         4.5       0.00      0.00      0.00        20
         5.0       0.00      0.00      0.00         3

    accuracy                           0.26       783
   macro avg       0.14      0.13      0.13       783
weighted avg       0.26      0.26      0.26       783



In [ ]:
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

svm_classifier = make_pipeline(StandardScaler(with_mean=False), LinearSVC(random_state=0, tol=1e-5, max_iter=4000))
svm_classifier.fit(tfidf_train, train_labels)
y_pred = svm_classifier.predict(tfidf_test)
print(classification_report(test_labels, y_pred, target_names=target_names_list))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         1.5       0.00      0.00      0.00         6
         2.0       0.17      0.09      0.11        82
         2.5       0.27      0.26      0.26       168
         3.0       0.33      0.38      0.35       250
         3.5       0.24      0.32      0.27       174
         4.0       0.16      0.12      0.13        78
         4.5       0.00      0.00      0.00        20
         5.0       0.00      0.00      0.00         3

    accuracy                           0.27       783
   macro avg       0.13      0.13      0.13       783
weighted avg       0.25      0.27      0.25       783



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.linear_model import LogisticRegression

logistic_regression_classifier = LogisticRegression(random_state=0).fit(tfidf_train, train_labels)

y_pred = logistic_regression_classifier.predict(tfidf_test)
print(classification_report(test_labels, y_pred, target_names=target_names_list))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         2
         1.5       0.00      0.00      0.00         6
         2.0       0.50      0.04      0.07        82
         2.5       0.28      0.21      0.24       168
         3.0       0.34      0.65      0.44       250
         3.5       0.30      0.29      0.30       174
         4.0       0.00      0.00      0.00        78
         4.5       0.00      0.00      0.00        20
         5.0       0.00      0.00      0.00         3

    accuracy                           0.32       783
   macro avg       0.16      0.13      0.12       783
weighted avg       0.29      0.32      0.27       783



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

# Try using word2vec

In [ ]:
import gensim.downloader as api
from gensim.test.utils import datapath
from gensim import utils
import gensim

model = api.load('word2vec-google-news-300') #load in the google model from the gensim library

[================----------------------------------] 32.7% 543.2/1662.8MB downloaded

KeyboardInterrupt: ignored

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=2, shuffle=True)
train_labels = train['syntax'].values.astype(int).tolist()
test_labels = test['syntax'].values.astype(int).tolist()


In [ ]:
train_sentences = []
word2vec_train_labels = []

# take each word in a review and average the word2vec vector 
for count, row in enumerate(train['full_text'].iteritems()):
    tokenized_words = word_tokenize(row[-1])
    words = []
    for w in tokenized_words:
        if w in model.key_to_index:
            word_vec = model.get_vector(w)
            words.append(word_vec)

    average_vals = np.average(words, axis=0)

    if average_vals.size == 300:
        # add averaged vector to train_sentences for training
        train_sentences.append(average_vals) 
        word2vec_train_labels.append(train_labels[count]) # get label -1 since model classifies 0-4 instead of 1-5


In [ ]:
print("size of train sentences: ", len(train_sentences))
print("size of train labels: ", len(word2vec_train_labels))

In [ ]:
test_sentences = []
word2vec_test_labels = []

# take each word from test set reviews and average the word2vec vector
for count, row in enumerate(test['full_text'].iteritems()):
    tokenized_words = word_tokenize(row[-1])
    words = []
    for w in tokenized_words:
        if w in model.key_to_index:
            word_vec = model.get_vector(w)
            words.append(word_vec)
    
    average_vals = np.average(words, axis=0)
    if average_vals.size == 300:
        test_sentences.append(average_vals)
        word2vec_test_labels.append(test_labels[count])

In [ ]:
print("size of train sentences: ", len(test_sentences))
print("size of train labels: ", len(word2vec_test_labels))
print(word2vec_test_labels[0:20])

In [ ]:
# build the train and test lists for the fnn model
word2vec_train_labels_adjusted = [x - 2 for x in word2vec_train_labels]
word2vec_test_labels_adjusted = [x - 2 for x in word2vec_test_labels]
train_list = list(zip(train_sentences, word2vec_train_labels_adjusted))
test_list = list(zip(test_sentences, word2vec_test_labels_adjusted))

In [ ]:
perceptron_classifier.fit(train_sentences, word2vec_train_labels)
y_pred = perceptron_classifier.predict(test_sentences)

print("Perceptron class accuracy results")
target_names = ['class 1', 'class 2', 'class 3', 'class 4', 'class 5', 'class 6', 'class 7', 'class 8', 'class 9']
print(classification_report(word2vec_test_labels, y_pred, target_names=target_names_list, digits=8))

In [ ]:
from sklearn.svm import SVC

svm_classifier = make_pipeline(StandardScaler(with_mean=False), SVC(kernel ='rbf', random_state=0, tol=1e-5, max_iter=4000))

svm_classifier.fit(train_sentences, word2vec_train_labels)
y_pred = svm_classifier.predict(test_sentences)

print("SVM class accuracy results")
print(classification_report(word2vec_test_labels, y_pred, target_names=target_names_list, digits=8))


In [ ]:
logistic_regression_classifier = LogisticRegression(random_state=0).fit(train_sentences, word2vec_train_labels)

y_pred = logistic_regression_classifier.predict(test_sentences)
print(classification_report(word2vec_test_labels, y_pred, target_names=target_names_list, digits=8))

# Deep learning on word2vec

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# define the NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(300, 50),
            nn.ReLU(),
            nn.Linear(50, 16),
            nn.ReLU(),
            nn.Linear(16, 9)
        )

    def forward(self, x):
        return self.layers(x)
# initialize the NN
model_nn = Net()
print(model_nn)
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer = torch.optim.SGD(model_nn.parameters(), lr=0.005)

In [ ]:
# number of subprocesses to use for data loading
from random import shuffle


num_workers = 0
# how many samples per batch to load
batch_size = 8

# obtain training indices that will be used for validation
num_train = len(train_sentences)
indices = list(range(num_train))
np.random.shuffle(indices)

valid_size = 0.2
# obtain training indices that will be used for validation
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders

train_loader = torch.utils.data.DataLoader(train_list, batch_size=batch_size,
     sampler=train_sampler, num_workers=num_workers)

valid_loader = torch.utils.data.DataLoader(train_list, batch_size=batch_size,
     sampler=valid_sampler, num_workers=num_workers)

test_loader = torch.utils.data.DataLoader(test_list, batch_size=batch_size, 
    num_workers=num_workers)

In [ ]:
for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0 #use validation to gage overfitting and best epoch length
    counter = 0 # use to keep track of number of occurrences

    # train the model 
    model_nn.train() # prep model for training
    for idx, (data,target) in enumerate(train_loader):
        counter += 1

        # clear the gradients of all optimized variables       
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model_nn(data)
        # calculate the loss
        # print(target.shape)
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
        if counter%2000 == 0 or counter == len(train_loader):
            #if a print encounter, run the validation set to determine error
            for data, target in valid_loader:
                # forward pass: compute predicted outputs by passing inputs to the model
                output = model_nn(data)
                # calculate the loss
                loss = criterion(output, target)
                # update running validation loss 
                valid_loss += loss.item()*data.size(0)
            
            valid_loss = valid_loss/len(valid_loader.dataset)
            print("Epoch {}...... Step: {}/{}....... Average Loss for Epoch: {},  Average Loss Overall: {}, Validation Loss: {}"
                  .format(epoch, counter, len(train_loader), train_loss/counter, train_loss/len(train_loader.dataset), valid_loss))
        



In [ ]:

#build predict model for test set
def predict(model_nn, dataloader, y_true):
    prediction_list = []
    for i, batch in enumerate(dataloader):
        # print(batch)
        data = torch.tensor(batch[0])   
        outputs = model_nn(data)
        _, predicted = torch.max(outputs.data, 1) 
        prediction_list.append(predicted.cpu())
        y_true.append(batch[1])
    return prediction_list


#build accuracy model for FNN
def accuracy(model_nn, predictions, y_true, batch, test_size):
    correct = 0
    # print(len(predictions))
    for i in range(0, len(predictions)):
        curBatch = predictions[i]
        # print(curBatch)
        curLabels = y_true[i]
        
        for j, pred in enumerate(curBatch):
            # print(pred, curLabels[j])           
            if pred == curLabels[j]:
                correct += 1
        # print(correct)
    
    print(correct)
    accuracy = correct / test_size
    return accuracy

In [ ]:
test_size = len(test_sentences)

print("test size: ", test_size)
print("size of test sentences: ", len(test_sentences))

y_true = []
predictions = predict(model_nn,test_loader, y_true)

print(len(predictions))
print(predictions[0].shape)
print(len(y_true))

print(accuracy(model_nn, predictions, y_true, 20, test_size))

# Try XLNet BERT Model

In [9]:
!pip install Sentencepiece
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [74]:
!pip install keras
!pip install tensorflow
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
# from transformers import XLNetTokenizer, XLNetModel
# https://medium.com/@yingbiao/text-classification-with-xlnet-in-action-869029246f7e
#  http://restanalytics.com/2021-05-04-Fine-Tuning-XLNet-For-Sequence-Classification/
from transformers import XLNetTokenizer, XLNetForMultipleChoice, XLNetForSequenceClassification
import torch

xlnet_model_name = 'xlnet-base-cased'
tokenizer = XLNetTokenizer.from_pretrained(xlnet_model_name, do_lower_case=True)

print(len(target_names_list))
print(target_names_list)
# model = XLNetForMultipleChoice.from_pretrained(xlnet_model_name, num_labels=len(target_names_list))
model = XLNetForSequenceClassification.from_pretrained(xlnet_model_name, num_labels=len(target_names_list))


9
['1.0', '1.5', '2.0', '2.5', '3.0', '3.5', '4.0', '4.5', '5.0']


Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [19]:
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
from torch import nn
class XLNetModel(nn.Module):
    
    def __init__(self, dropout=0.1):
        
        super(XLNetModel, self).__init__()
        self.xlnet =  AutoModel.from_pretrained('xlnet-base-cased')
        self.dropout = nn.Dropout(dropout,0)
        self.linear = nn.Linear(768,256)
        self.relu = nn.LeakyReLU(0.1)
        self.linear_2 = nn.Linear(256, 128)
        self.out = nn.Linear(128,1)
        
        
    def forward(self,input_id,mask):
        x = self.xlnet(input_ids=input_id,attention_mask=mask,return_dict=False)
        # print(x[0].shape)

        x = self.dropout(x[0])
        x = self.linear(x)
        x = self.relu(x)
        x = self.linear_2(x)
        x = self.relu(x)
        final_layer = self.out(x)
        # print(final_layer.shape)
        return final_layer

In [20]:
model = XLNetModel()
device = torch.device("cuda")
model.to(device)
print()

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetModel: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
# try https://mccormickml.com/2019/09/19/XLNet-fine-tuning/
from keras.utils import pad_sequences
from nltk import tokenize
xlnet_train_list = []
xlnet_train_labels = []
attention_masks_train = []
num_embeddings = 160
SEP = "[SEP]"
CLS = "[CLS]"


for count, txt in enumerate(train['full_text']):
  
  sentences = tokenize.sent_tokenize(txt)
  for i, sentence in enumerate(sentences):
    if i == len(sentences):
      sentences[i] = sentence + SEP + " " + CLS
    else:
      sentences[i] = sentence + SEP
  
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  # if count == 0:
  #   print ("Tokenize the first sentence:")
  #   print (tokenized_texts[0])
  
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post").squeeze()

  # tokens = tokenizer.encode(txt, max_length=512, add_special_tokens=False)
  # tokens = tokenizer.tokenize(txt)
  # input_ids = [tokenizer.convert_tokens_to_ids(tokens)[:num_embeddings-2]]
  # input_ids = [tokenizer.build_inputs_with_special_tokens(input_ids)]
  # input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post")
  attention_masks = []
  # token_lens.append(len(tokens))
  for seq in input_ids:
    # seq_mask = [float(s>0) for s in seq]
    attention_masks.append(float(seq>0))
  
  if count == 0:
  #   print ("Tokenize the first sentence:")
    print(input_ids.shape)
    print(input_ids)
    print(attention_masks)
  
  xlnet_train_list.append(input_ids)
  xlnet_train_labels.append(float(train_labels[count] -2))
  attention_masks_train.append((attention_masks))

# print(xlnet_train_labels)

(160,)
[   17  6684  3742    17  2554  8037   672  1143  2081   287 17949  3239
   178   132   588   426  3617   451    23  3079    93   179   359   206
    92   730  8686   451    23  3079    93   730   634 17949  5465  1125
 23415   451    23  3079    56   911   731  1826  3424  8686  1667  6274
    25 18135   235  1613  1872   235  1129  3697 27273  9160   451    23
  3079    56   104   451    23  3079    93  8686  1667  6274   426   239
  3110  1166   352  1129  2598   115 10883   530  5953   150   371  6274
   401   185  1795 10871   121  1772 15609   426 24255   952   945  1433
   332  1251  1104   507   507  1337 16445  8686    56  2986  1939   185
  1075  4549   928   494    65   185    23   252 17408  4549   793   185
   494  1337   176  1000   977   451    23  3079    93  1667  6274   293
  1194   331 20601   459   235  3704   197   178   588    17   775   129
    87  2904   123 19764   913  2177   199 17949   426    17  9462  3079
    56    17  2996 11381]
[1.0, 1.0, 1.0, 1.

In [27]:
# test on test set and get accuracy
xlnet_test_list = []
xlnet_test_labels = []
attention_masks_test = []

for count, txt in enumerate(test['full_text']):
  
  sentences = tokenize.sent_tokenize(txt)
  for i, sentence in enumerate(sentences):
    if i == len(sentences):
      sentences[i] = sentence + SEP + " " + CLS
    else:
      sentences[i] = sentence + SEP
  
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  
  input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
  input_ids = pad_sequences(input_ids, maxlen=num_embeddings, dtype="long", truncating="post", padding="post").squeeze()

  attention_masks = []
  # token_lens.append(len(tokens))
  for seq in input_ids:
    # seq_mask = [float(s>0) for s in seq]
    attention_masks.append(float(seq>0))

  if count == 0:
    print(input_ids.shape)
    print(input_ids)
    print(attention_masks)

  xlnet_test_list.append(input_ids)
  xlnet_test_labels.append(test_labels[count]-2)
  attention_masks_test.append(attention_masks)
print(xlnet_test_labels)

(160,)
[  781  3188   195   781 10202   300   466  1230  1190  7083  1705    92
   467  1720   781  1304  1190  6439   781   673   679   466  2065 10202
  6439   781   466   132   434  4418  1184   332 12247   332   770  1785
  2032  4113   781  1304 10202   634   312   875   466  6439   781   319
   679   312   875   466  6439   781   673   679  1190   175  1037  3309
   978  1244   781   466  3894 10202   466   551  1627  1949  4642    23
  1924   790   623   781  3097   312   875   466  6439   781   673   679
   466   175  1037  3309   978  1244   781  1190   589  1437   781   426
   312   875  1078   310  2550  3219  3659  1872   144   955 27927  2589
  2737  1667   343   955   466  1250  3309   427   175   214  2383  1194
  4277  1080  3431  1190   132  1578   133   383   920  4954  5940   790
    41    60  8566  1767  1213  1556   920  1058   920  5476  3659   312
  2141  1190    74   312   994  1190  6439   781   673   679  3272   175
  5940   790  1244   781]
[1.0, 1.0, 1.0, 1.

In [35]:
# loop through train_list and run text on tokenizer
attention_masks_train = torch.tensor(attention_masks_train)
print(len(xlnet_train_list))
print(len(xlnet_train_labels))
print(xlnet_train_labels)
xlnet_tokenized_train = list(zip(xlnet_train_list, xlnet_train_labels, attention_masks_train))

attention_masks_test = torch.tensor(attention_masks_test)
xlnet_test_labels = torch.tensor(xlnet_test_labels)
print(len(xlnet_test_list), len(xlnet_test_labels))
xlnet_tokenized_test = list(zip(xlnet_test_list, xlnet_test_labels, attention_masks_test))

3128
3128
[5.0, 5.0, 3.0, 3.0, 5.0, 4.0, 3.0, 4.0, 3.0, 6.0, 4.0, 2.0, 4.0, 3.0, 4.0, 4.0, 3.0, 6.0, 3.0, 4.0, 4.0, 5.0, 4.0, 3.0, 6.0, 3.0, 3.0, 5.0, 2.0, 3.0, 3.0, 5.0, 3.0, 5.0, 3.0, 5.0, 5.0, 3.0, 4.0, 5.0, 2.0, 5.0, 3.0, 5.0, 6.0, 4.0, 5.0, 4.0, 4.0, 5.0, 4.0, 2.0, 5.0, 4.0, 5.0, 5.0, 5.0, 6.0, 5.0, 4.0, 4.0, 7.0, 6.0, 5.0, 5.0, 6.0, 3.0, 5.0, 4.0, 3.0, 3.0, 4.0, 6.0, 5.0, 5.0, 4.0, 4.0, 5.0, 7.0, 6.0, 5.0, 5.0, 4.0, 5.0, 5.0, 2.0, 4.0, 4.0, 5.0, 4.0, 3.0, 5.0, 3.0, 5.0, 3.0, 4.0, 3.0, 3.0, 3.0, 4.0, 2.0, 7.0, 4.0, 4.0, 5.0, 4.0, 3.0, 6.0, 3.0, 5.0, 2.0, 2.0, 6.0, 5.0, 4.0, 4.0, 5.0, 3.0, 5.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, 2.0, 4.0, 4.0, 4.0, 5.0, 6.0, 2.0, 2.0, 4.0, 3.0, 4.0, 5.0, 4.0, 5.0, 5.0, 5.0, 4.0, 5.0, 3.0, 4.0, 4.0, 3.0, 4.0, 5.0, 3.0, 3.0, 5.0, 4.0, 5.0, 2.0, 5.0, 5.0, 4.0, 4.0, 4.0, 4.0, 4.0, 4.0, 3.0, 6.0, 4.0, 6.0, 3.0, 3.0, 5.0, 5.0, 4.0, 4.0, 6.0, 4.0, 5.0, 5.0, 6.0, 5.0, 4.0, 7.0, 4.0, 6.0, 4.0, 4.0, 5.0, 4.0, 4.0, 6.0, 3.0, 4.0, 2.0, 3.0, 3.0, 5.0, 4.0, 5.0,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':


In [36]:

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
batch_size = 17
param_optimizer = list(model.named_parameters())
criterion = nn.MSELoss().to(device)

train_dataloader = DataLoader(xlnet_tokenized_train, sampler=RandomSampler(xlnet_tokenized_train), batch_size=batch_size)
val_dataloader = torch.utils.data.DataLoader(xlnet_tokenized_test, batch_size=batch_size)
# for data in train_dataloader:
#   print(data)
# model.train()

In [37]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [39]:

epochs = 3
# print(len(train_dataloader))
for i in range(epochs):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    model.train()
    t0 = time.time()
    training_time = format_time(t0)
    for idx, (batch, target, mask) in enumerate(train_dataloader):
        
        input_ids_np = batch.to(device)
        # print(input_ids_np.shape)
        mask = mask.to(device)
        # print(mask.shape)
        target = target.to(device)
        # print(target.shape)
        # input_ids_np = torch.as_tensor(batch).unsqueeze(0)
        # mask = torch.tensor(mask, dtype=torch.int)

        optimizer.zero_grad()
        # outputs = model(input_ids=input_ids_np, attention_mask=mask, labels=target)
        output = model(input_ids_np, mask)
        # loss = outputs[0]
        # loss = criterion(outputs[0], target.float())
        loss = criterion(output, target.float())

        # print(loss.item())
        loss.backward()

        tr_loss += loss.item()
        # print(tr_loss, " is tr_loss")
        # print()
        # nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        optimizer.step()
        if idx > 0 and (idx%10 == 0 or idx == len(train_dataloader)):
            print("Epoch {}...... Step: {}/{}....... Average Loss for Epoch: {},  {}".format(i+1, idx, len(train_dataloader), tr_loss/idx, tr_loss/len(train_dataloader)))
    
    total_loss_val = 0

    with torch.no_grad():

        for idx, (val_input, val_label, val_mask) in enumerate(val_dataloader):

            val_label = val_label.to(device)
            mask = val_mask.to(device)
            input_id = val_input.squeeze(1).to(device)

            output = model(input_id, mask)

            loss = criterion(output, val_label)
            total_loss_val += loss.item()

            if idx > 0 and (idx%10 == 0 or idx == len(train_dataloader)):
              print("Epoch {}...... Step: {}/{}....... Average Loss for Epoch: {},  {}".format(i+1, idx, len(train_dataloader), tr_loss/idx, tr_loss/len(train_dataloader)))
    
    
    epoch_train_loss = tr_loss/len(train_dataloader)
    training_time = format_time(time.time() - t0)
    print("")
    print("Overall for Epoch {}: Average training loss: {} Average validation loss: {}".format(i+1, epoch_train_loss, total_loss_val / len(val_dataloader)))
    print("  Training epoch took: {}".format(training_time))
    print("")
    # 

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([17])) that is different to the input size (torch.Size([17, 160, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1...... Step: 10/184....... Average Loss for Epoch: 1.8887580394744874,  0.1026498934497004
Epoch 1...... Step: 20/184....... Average Loss for Epoch: 1.901164823770523,  0.20664835040983948
Epoch 1...... Step: 30/184....... Average Loss for Epoch: 1.906394612789154,  0.3108252086069273
Epoch 1...... Step: 40/184....... Average Loss for Epoch: 1.7919228821992874,  0.389548452652019
Epoch 1...... Step: 50/184....... Average Loss for Epoch: 1.8108781254291535,  0.49208644712748734
Epoch 1...... Step: 60/184....... Average Loss for Epoch: 1.7967339744170507,  0.5858915133968644
Epoch 1...... Step: 70/184....... Average Loss for Epoch: 1.7686150644506726,  0.6728426875627559
Epoch 1...... Step: 80/184....... Average Loss for Epoch: 1.7958178482949734,  0.7807903688239015
Epoch 1...... Step: 90/184....... Average Loss for Epoch: 1.8118633508682251,  0.8862375085768492
Epoch 1...... Step: 100/184....... Average Loss for Epoch: 1.810325837135315,  0.9838727375735408
Epoch 1...... Step: 1

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 160, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)



Overall for Epoch 1: Average training loss: 1.747471056068721 Average validation loss: 1.6913234704035394
  Training epoch took: 0:02:11

Epoch 2...... Step: 10/184....... Average Loss for Epoch: 1.972952628135681,  0.1072256863117218
Epoch 2...... Step: 20/184....... Average Loss for Epoch: 1.892375648021698,  0.20569300521974979
Epoch 2...... Step: 30/184....... Average Loss for Epoch: 1.8164030631383261,  0.29615267333777057
Epoch 2...... Step: 40/184....... Average Loss for Epoch: 1.748169380426407,  0.3800368218318276
Epoch 2...... Step: 50/184....... Average Loss for Epoch: 1.8083932995796204,  0.49141122271185333
Epoch 2...... Step: 60/184....... Average Loss for Epoch: 1.8074173510074616,  0.589375223154607
Epoch 2...... Step: 70/184....... Average Loss for Epoch: 1.7654887122767313,  0.6716533144531043
Epoch 2...... Step: 80/184....... Average Loss for Epoch: 1.788706538826227,  0.7776984951418379
Epoch 2...... Step: 90/184....... Average Loss for Epoch: 1.7972088674704234,  

In [26]:
from sklearn.metrics import mean_squared_error
def mcr_mse(y_trues, y_preds):
    scores = []
    idxes = y_trues.shape[1]
    for i in range(idxes):
        y_true = y_trues[:,i]
        y_pred = y_preds[:,i]
        score = mean_squared_error(y_true, y_pred, squared=False)
        scores.append(score)
    mcrmse_score = np.mean(scores)
    return mcrmse_score, scores

In [28]:

attention_masks_test = torch.tensor(attention_masks_test)
xlnet_test_labels = torch.tensor(xlnet_test_labels)
print(len(xlnet_test_list), len(xlnet_test_labels))
xlnet_tokenized_test = list(zip(xlnet_test_list, xlnet_test_labels, attention_masks_test))

783 783


In [30]:

test_dataloader = DataLoader(xlnet_tokenized_test, sampler=RandomSampler(xlnet_tokenized_test), batch_size=batch_size)

model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for idx, (batch,target, mask) in enumerate(test_dataloader):
  input_ids_np = batch.to(device)
  target = target.to(device)
  mask = mask.to(device)
  with torch.no_grad():
    # print(target)
    # outputs = model(input_ids=input_ids_np, attention_mask=mask, labels=target)
    output = model(input_ids_np, mask)
    print(output)
    # logits = outputs[1]
    # print(logits)
    # mcrmse = mcr_mse(outputs[1].cpu(), target.cpu())
    mcrmse = mcr_mse(output, target)
    print(mcrmse)
  # logits = logits.sigmoid().detach().cpu().numpy()
  # label_ids = target.cpu().numpy()
  predictions.append(outputs)
  true_labels.append(label_ids)

tensor([[[3.5435],
         [3.5435],
         [3.5435],
         ...,
         [3.5435],
         [3.5435],
         [3.5435]],

        [[3.5435],
         [3.5435],
         [3.5435],
         ...,
         [3.5435],
         [3.5435],
         [3.5435]],

        [[3.5435],
         [3.5435],
         [3.5435],
         ...,
         [3.5435],
         [3.5435],
         [3.5435]],

        ...,

        [[3.5435],
         [3.5435],
         [3.5435],
         ...,
         [3.5435],
         [3.5435],
         [3.5435]],

        [[3.5435],
         [3.5435],
         [3.5435],
         ...,
         [3.5435],
         [3.5435],
         [3.5435]],

        [[3.5435],
         [3.5435],
         [3.5435],
         ...,
         [3.5435],
         [3.5435],
         [3.5435]]], device='cuda:0')


IndexError: ignored

In [ ]:
print(predictions[0])
predictions = torch.tensor(predictions)
true_labels = torch.tensor(true_labels)
print(predictions, true_labels)
mcr_mse(predictions, true_labels)

XLNetForSequenceClassificationOutput(loss=tensor(3.8802, device='cuda:0'), logits=tensor([[ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462,  2.2846,  2.0155,  0.9988, -0.7767, -2.0933, -2.9940, -3.5153,
         -1.3148],
        [ 1.8462, 

RuntimeError: ignored

In [ ]:
from sklearn.metrics import matthews_corrcoef
matthews_set = []

for i in range(len(true_labels)):
  print(predictions[i], " is prediction")
  print(true_labels)
  matthews = matthews_corrcoef(true_labels[i],
                 np.argmax(predictions[i], axis=1).flatten())
  print(matthews)
  matthews_set.append(matthews)

In [ ]:
flat_predictions = [item for sublist in predictions for item in sublist]
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = [item for sublist in true_labels for item in sublist]


In [ ]:
matthews_corrcoef(flat_true_labels, flat_predictions)